<a href="https://colab.research.google.com/github/kaisu313/Capstone-Week-1-submission/blob/master/Vancouver_Restaurant_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **PROBLEM/BUSINESS CASE:  
A friend just immigrated from Taiwan and would like to setup a new dessert shop in downtown vancouver where he lives.  He currently has restaurants in Taiwan so its really only a matter of picking the right place for his first shop in Vancouver.  

After months of searching, he has narrowed down to downtown vancouver South West vs North East area.  Both areas have similar demographics so now it is up to which area will have less competition in that space.  The target audience of his store will be general public living and working in downtown vancouver.  It is important that this business thrives as it is his first flagship shop in Canada.  Future prospects for further expansion will hinge on whether the store is successful.

## **DATA DESCRIPTION
Based on the above criteria, and tO help him find which area has less competition, we will use the following data sources:

1) Vancouver BC's location coordinates through GEONAMES
 https://www.geonames.org/postal-codes/CA/BC/british-columbia.html
 
 We will scrub this page and put the relevant datatable into a dataframe.  
 This data is discrete attribute data which consists of place, postal code, name of neighborhood, province and country.  We will use this data's coordinates and location data as an input into Foursquare's API call to explore the neighborhood and venues.  


 2)  FourSquare to explore the most common venues in each area

 We will extract the venues in each area through calling FourSquare's API. This dataset return will contain the type of business, as well as its popularity.  With this attribute data set, we can rank them accordingly to obtain the most commnon venue within the neighborhood.


 3)  We will compare locate South West downtown and North East downtown to see which area have less competition (i.e. serving desserts)


